In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/riiid-test-answer-prediction/lectures.csv
/kaggle/input/riiid-test-answer-prediction/example_test.csv
/kaggle/input/riiid-test-answer-prediction/example_sample_submission.csv
/kaggle/input/riiid-test-answer-prediction/train.csv
/kaggle/input/riiid-test-answer-prediction/questions.csv
/kaggle/input/riiid-test-answer-prediction/riiideducation/__init__.py
/kaggle/input/riiid-test-answer-prediction/riiideducation/competition.cpython-37m-x86_64-linux-gnu.so


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import lightgbm as lgb
import xgboost as xgb

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV

In [3]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

### Data

In [4]:
train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv', 
                       low_memory=False, 
                       nrows=10**6, 
                       dtype={'row_id': 'int64', 
                              'timestamp': 'int64', 
                              'user_id': 'int32', 
                              'content_id': 'int16', 
                              'content_type_id': 'int8',
                              'task_container_id': 'int16', 
                              'user_answer': 'int8', 
                              'answered_correctly': 'int8', 
                              'prior_question_elapsed_time': 'float32', 
                              'prior_question_had_explanation': 'boolean'
                             }
                      )

print(train_df.shape)
train_df.head()

(1000000, 10)


,row_id,timestamp,user_id,content_id,content_type_id,task_container_id,user_answer,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
0,0,0,115,5692,0,1,3,1,NaN,<NA>
1,1,56943,115,5716,0,2,2,1,37000.0,False
2,2,118363,115,128,0,0,0,1,55000.0,False
3,3,131167,115,7860,0,3,0,1,19000.0,False
4,4,137965,115,7922,0,4,1,1,11000.0,False


In [5]:
print('Part of missing values for every column')
print(train_df.isnull().sum() / len(train_df))

Part of missing values for every column
row_id                            0.000000
timestamp                         0.000000
user_id                           0.000000
content_id                        0.000000
content_type_id                   0.000000
task_container_id                 0.000000
user_answer                       0.000000
answered_correctly                0.000000
prior_question_elapsed_time       0.023723
prior_question_had_explanation    0.003816
dtype: float64


In [6]:
questions = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/questions.csv')

print(questions.shape)
questions.head()

(13523, 5)


,question_id,bundle_id,correct_answer,part,tags
0,0,0,0,1,51 131 162 38
1,1,1,1,1,131 36 81
2,2,2,0,1,131 101 162 92
3,3,3,0,1,131 149 162 29
4,4,4,3,1,131 5 162 38


In [7]:
print('Part of missing values for every column')
print(questions.isnull().sum() / len(questions))

Part of missing values for every column
question_id       0.000000
bundle_id         0.000000
correct_answer    0.000000
part              0.000000
tags              0.000074
dtype: float64


In [8]:
lectures = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/lectures.csv')

print(lectures.shape)
lectures.head()

(418, 4)


,lecture_id,tag,part,type_of
0,89,159,5,concept
1,100,70,1,concept
2,185,45,6,concept
3,192,79,5,solving question
4,317,156,5,solving question


In [9]:
print('Part of missing values for every column')
print(lectures.isnull().sum() / len(lectures))

Part of missing values for every column
lecture_id    0.0
tag           0.0
part          0.0
type_of       0.0
dtype: float64


In [10]:
example_test = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/example_test.csv')

print(example_test.shape)
example_test.head()

(104, 11)


,row_id,group_num,timestamp,user_id,content_id,content_type_id,task_container_id,prior_question_elapsed_time,prior_question_had_explanation,prior_group_answers_correct,prior_group_responses
0,0,0,0,275030867,5729,0,0,NaN,NaN,[],[]
1,1,0,13309898705,554169193,12010,0,4427,19000.0,True,NaN,NaN
2,2,0,4213672059,1720860329,457,0,240,17000.0,True,NaN,NaN
3,3,0,62798072960,288641214,13262,0,266,23000.0,True,NaN,NaN
4,4,0,10585422061,1728340777,6119,0,162,72400.0,True,NaN,NaN


In [11]:
print('Part of missing values for every column')
print(example_test.isnull().sum() / len(lectures))

Part of missing values for every column
row_id                            0.000000
group_num                         0.000000
timestamp                         0.000000
user_id                           0.000000
content_id                        0.000000
content_type_id                   0.000000
task_container_id                 0.000000
prior_question_elapsed_time       0.002392
prior_question_had_explanation    0.002392
prior_group_answers_correct       0.239234
prior_group_responses             0.239234
dtype: float64


In [12]:
used_data_types_dict = {
    'timestamp': 'int64',
    'user_id': 'int32',
    'content_id': 'int16',
    'answered_correctly': 'int8',
    'prior_question_elapsed_time': 'float16',
    'prior_question_had_explanation': 'boolean'
}


train_df = pd.read_csv('/kaggle/input/riiid-test-answer-prediction/train.csv',
                       usecols = used_data_types_dict.keys(),
                       dtype=used_data_types_dict, 
                       index_col = 0
)

print(train_df.shape)
train_df.head()

/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(101230332, 5)


,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
0,115,5692,1,NaN,<NA>
56943,115,5716,1,36992.0,False
118363,115,128,1,55008.0,False
131167,115,7860,1,19008.0,False
137965,115,7922,1,11000.0,False


### Create new features

In [13]:
features_df = train_df.iloc[:int(9 /10 * len(train_df))]
train_df = train_df.iloc[int(9 /10 * len(train_df)):]

In [14]:
print(train_df.shape)
train_df.head()

(10123034, 5)


,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
8231964660,1933715875,11259,0,13000.0,True
8232002976,1933715875,4957,1,44000.0,True
8232096407,1933715875,5113,1,22000.0,True
8232119872,1933715875,4699,1,inf,True
8232142930,1933715875,11430,1,9000.0,True


In [15]:
print(features_df.shape)
features_df.head()

(91107298, 5)


,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
0,115,5692,1,NaN,<NA>
56943,115,5716,1,36992.0,False
118363,115,128,1,55008.0,False
131167,115,7860,1,19008.0,False
137965,115,7922,1,11000.0,False


In [16]:
train_questions_only_df = features_df[features_df['answered_correctly']!=-1]

print(train_questions_only_df.shape)
train_questions_only_df.head()

(89344349, 5)


,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
0,115,5692,1,NaN,<NA>
56943,115,5716,1,36992.0,False
118363,115,128,1,55008.0,False
131167,115,7860,1,19008.0,False
137965,115,7922,1,11000.0,False


In [17]:
# user_id
grouped_by_user_df = train_questions_only_df.groupby('user_id')

grouped_by_user_df.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
0,115,5692,1,NaN,<NA>
56943,115,5716,1,36992.0,False
118363,115,128,1,55008.0,False
131167,115,7860,1,19008.0,False
137965,115,7922,1,11000.0,False
...,...,...,...,...,...
0,1933715875,4332,1,NaN,<NA>
39339,1933715875,4217,1,24000.0,False
76378960,1933715875,98,0,33984.0,False


In [18]:
user_answers_df = grouped_by_user_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew']}).copy()

user_answers_df.columns = ['mean_user_accuracy', 
                           'questions_answered', 
                           'std_user_accuracy', 
                           'median_user_accuracy', 
                           'skew_user_accuracy']

user_answers_df.head()

,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy
user_id,,,,,
115,0.695652,46,0.465215,1.0,-0.879359
124,0.233333,30,0.430183,0.0,1.328338
2746,0.578947,19,0.507257,1.0,-0.347892
5382,0.672000,125,0.471374,1.0,-0.741648
8623,0.642202,109,0.481566,1.0,-0.601619


In [19]:
# content_id
grouped_by_content_df = train_questions_only_df.groupby('content_id')

grouped_by_content_df.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation
timestamp,,,,,
0,115,5692,1,NaN,<NA>
56943,115,5716,1,36992.0,False
118363,115,128,1,55008.0,False
131167,115,7860,1,19008.0,False
137965,115,7922,1,11000.0,False
...,...,...,...,...,...
2210872852,1101821012,3572,1,11000.0,True
1286282597,1333688829,10033,1,35008.0,True
2533051536,1576785630,3557,0,26000.0,True


In [20]:
content_answers_df = grouped_by_content_df.agg({'answered_correctly': ['mean', 'count', 'std', 'median', 'skew']}).copy()

content_answers_df.columns = ['mean_accuracy', 
                              'question_asked', 
                              'std_accuracy',
                              'median_accuracy', 
                              'skew_accuracy']

content_answers_df

,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy
content_id,,,,,
0,0.908595,6236,0.288207,1.0,-2.836339
1,0.891682,6684,0.310805,1.0,-2.521185
2,0.554656,40499,0.497010,1.0,-0.219949
3,0.779348,20734,0.414696,1.0,-1.347371
4,0.613226,28549,0.487020,1.0,-0.465009
...,...,...,...,...,...
13518,0.789203,778,0.408137,1.0,-1.420839
13519,0.567797,826,0.495682,1.0,-0.274213
13520,0.678524,759,0.467351,1.0,-0.766003


In [21]:
features = [
    'mean_user_accuracy', 
    'questions_answered',
    'std_user_accuracy', 
    'median_user_accuracy',
    'skew_user_accuracy',
    'mean_accuracy', 
    'question_asked',
    'std_accuracy', 
    'median_accuracy',
    'prior_question_elapsed_time', 
    'prior_question_had_explanation',
    'skew_accuracy'
]

target = 'answered_correctly'

In [22]:
train_df = train_df.merge(user_answers_df, how='left', on='user_id')
train_df = train_df.merge(content_answers_df, how='left', on='content_id')

train_df.head()

,user_id,content_id,answered_correctly,prior_question_elapsed_time,prior_question_had_explanation,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy,mean_accuracy,question_asked,std_accuracy,median_accuracy,skew_accuracy
0,1933715875,11259,0,13000.0,True,0.779843,5219.0,0.414392,1.0,-1.351136,0.532146,1291.0,0.499159,1.0,-0.128999
1,1933715875,4957,1,44000.0,True,0.779843,5219.0,0.414392,1.0,-1.351136,0.584772,2548.0,0.492858,1.0,-0.344273
2,1933715875,5113,1,22000.0,True,0.779843,5219.0,0.414392,1.0,-1.351136,0.603571,1960.0,0.489280,1.0,-0.423795
3,1933715875,4699,1,inf,True,0.779843,5219.0,0.414392,1.0,-1.351136,0.694888,2504.0,0.460547,1.0,-0.847011
4,1933715875,11430,1,9000.0,True,0.779843,5219.0,0.414392,1.0,-1.351136,0.765869,1922.0,0.423565,1.0,-1.256695


In [23]:
train_df['prior_question_had_explanation'] = train_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
train_df = train_df.fillna(value=0.5)

In [24]:
train_df = train_df[features + [target]]
train_df = train_df.replace([np.inf, -np.inf], np.nan)
train_df = train_df.fillna(0.5)

train_df.head()

,mean_user_accuracy,questions_answered,std_user_accuracy,median_user_accuracy,skew_user_accuracy,mean_accuracy,question_asked,std_accuracy,median_accuracy,prior_question_elapsed_time,prior_question_had_explanation,skew_accuracy,answered_correctly
0,0.779843,5219.0,0.414392,1.0,-1.351136,0.532146,1291.0,0.499159,1.0,13000.0,True,-0.128999,0
1,0.779843,5219.0,0.414392,1.0,-1.351136,0.584772,2548.0,0.492858,1.0,44000.0,True,-0.344273,1
2,0.779843,5219.0,0.414392,1.0,-1.351136,0.603571,1960.0,0.489280,1.0,22000.0,True,-0.423795,1
3,0.779843,5219.0,0.414392,1.0,-1.351136,0.694888,2504.0,0.460547,1.0,0.5,True,-0.847011,1
4,0.779843,5219.0,0.414392,1.0,-1.351136,0.765869,1922.0,0.423565,1.0,9000.0,True,-1.256695,1


### Modeling

In [25]:
params = {
    'bagging_fraction': 0.5817242323514327,
    'feature_fraction': 0.6884588361650144,
    'learning_rate': 0.42887924851375825, 
    'max_depth': 6,
    'min_child_samples': 946, 
    'min_data_in_leaf': 47, 
    'n_estimators': 169,
    'num_leaves': 29,
    'random_state': 666
}

model = lgb.LGBMClassifier(**params)
model.fit(train_df[features], train_df[target])

LGBMClassifier(bagging_fraction=0.5817242323514327,
               feature_fraction=0.6884588361650144,
               learning_rate=0.42887924851375825, max_depth=6,
               min_child_samples=946, min_data_in_leaf=47, n_estimators=169,
               num_leaves=29, random_state=666)

In [26]:
for (test_df, sample_prediction_df) in iter_test:
    test_df = test_df.merge(user_answers_df, how='left', on='user_id')
    test_df = test_df.merge(content_answers_df, how='left', on='content_id')
    test_df['prior_question_had_explanation'] = test_df['prior_question_had_explanation'].fillna(value=False).astype(bool)
    test_df.fillna(value = 0.5, inplace = True)

    test_df['answered_correctly'] = model.predict_proba(test_df[features])[:,1]
    env.predict(test_df.loc[test_df['content_type_id'] == 0, ['row_id', 'answered_correctly']])